<a href="https://colab.research.google.com/github/davidwhogg/synthetic_calibration/blob/main/ipynb/eboss_arcs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploratory data analysis with *SDSS-IV eBOSS* arc exposures

*by* **David W. Hogg**

## Notes:
- The most recent public repository of raw *BOSS* spectrograph data is at `"https://data.sdss.org/sas/dr16/eboss/spectro/data/{:05d}/".format(mjd)`
- The non-public repository of raw *BOSS* spectrograph data is at `"rsync://sdss@dtn01.sdss.org/sas/ebosswork/eboss/spectro/data/{:05d}/".format(mjd)`




In [ ]:
!pip install corner

In [ ]:
import numpy as np
import pylab as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.io import fits
import urllib
from urllib.request import Request, urlopen, urlretrieve
from bs4 import BeautifulSoup
import corner

In [ ]:
# STUPID AUTH SHIT
if False:
  # create a password manager
  password_mgr = urllib.request.HTTPPasswordMgrWithDefaultRealm()

  # Add the username and password.
  # If we knew the realm, we could use it instead of None.
  top_level_url = "https://data.sdss.org/sas/ebosswork/"
  password_mgr.add_password(None, top_level_url, "sdss", "censored") # WRONG PASSWORD
  handler = urllib.request.HTTPBasicAuthHandler(password_mgr)

  # create "opener" (OpenerDirector instance)
  opener = urllib.request.build_opener(handler)

  # Install the opener.
  # Now all calls to urllib.request.urlopen use our opener.
  urllib.request.install_opener(opener)

In [ ]:
# testing
if False:
  url = make_boss_data_url(58779)
  urls = get_all_sdR_urls(url)

In [ ]:
def make_boss_data_url(mjd):
  """
  Find a public directory of raw boss data, given an MJD.
  """
  # return "http://localhost:8888/tree/{:05d}/".format(mjd)
  # return "https://data.sdss.org/sas/ebosswork/eboss/spectro/data/{:05d}/".format(mjd)
  return "https://data.sdss.org/sas/dr16/eboss/spectro/data/{:05d}/".format(mjd)

def get_all_sdR_urls(mjd, camera="r1"):
  """
  Find all public sdR file urls, given a directory url.

  This code is overly specific!
  """
  url = make_boss_data_url(mjd)
  req = Request(url)
  a = urlopen(req).read()
  soup = BeautifulSoup(a, 'html.parser')
  x = (soup.find_all('a'))
  urls = []
  for i in x:
    file_name = i.extract().get_text()
    if "sdR-{}-".format(camera) in file_name:
      url_new = url + file_name
      # url_new = url_new.replace(" ","%20")
      urls.append(url_new)
  return np.unique(urls)

def grab_all_arcs(mjd, slitids=(14,), camera="r1"):
  """
  Find all arcs from one slithead on one mjd, from public data.

  ## Bugs:
  - Terrible >100. hack; I don't know what to do about these zero images?
  """
  arcs = []
  try:
    sdR_urls = get_all_sdR_urls(mjd, camera=camera)
  except:
    return arcs
  for url in sdR_urls:
    hdu = fits.open(url)
    hdr = hdu[0].header
    if ("arc" in hdr["FLAVOR"]) and \
     (hdr["SLITID1"] in slitids):
      assert hdr["SLITID1"] == hdr["SLITID2"]
      if np.median(hdu[0].data) > 100.: # HACK MAGIC HACK
        print("found arc", url)
        arcs.append((url, hdr, hdu[0].data))
      else:
        print("bad arc?", np.median(hdu[0].data), url)
        print(hdr)
    hdu.close()
    del hdu
  print("On {}, I found a total of {} arcs for slitheads {}.".format(mjd, len(arcs), slitids))
  return arcs

In [ ]:
arcs = []
for mjd in np.arange(58540, 58542):
  arcs += grab_all_arcs(mjd, camera="r1")

In [ ]:
print("total of {} arcs".format(len(arcs)))

In [ ]:
for url, hdr, image in arcs:
  print(url, image.shape, hdr["ALT"], hdr["AZ"], hdr["IPA"], hdr["EXPTIME"],
        np.min(image), np.median(image), np.max(image))

In [ ]:
with PdfPages('boss_arc_zoom.pdf') as pdf:
  for url, hdr, image in arcs:
    subimage = image[1700:1900, 1500:1700].astype(float)
    subimage -= np.median(subimage)
    fig = plt.figure(figsize=(12, 9))
    plt.imshow(subimage, vmin=-30, vmax=30.0,
               cmap="gray", interpolation="nearest")
    plt.colorbar()
    plt.title(url)
    pdf.savefig(fig)
    plt.close(fig)

In [ ]:
def _cc(dx, dy, image, reference, maxshift):
  tmp = np.roll(image, (dy, dx), axis=(0, 1))
  numerator = reference * tmp
  denominator = tmp * tmp
  return np.sum(numerator[maxshift:-maxshift,maxshift:-maxshift]) # / \
#       np.sum(denominator[maxshift:-maxshift,maxshift:-maxshift])

def cross_correlate(image, reference, maxshift=8):
  assert image.shape == reference.shape
  shifts = np.arange(-maxshift, maxshift + 1)
  ccf = np.zeros((len(shifts), len(shifts)))
  for ix, dx in enumerate(shifts):
    for iy, dy in enumerate(shifts):
      ccf[iy, ix] = _cc(dx, dy, image, reference, maxshift)
  return ccf, shifts

In [ ]:
def imslice(image):
  return (image[1700:1900, 1500:1700]).astype(float)

In [ ]:
def centroid_3(patch, xx=np.arange(-1, 2)):
  assert patch.shape == (3, )
  assert xx.shape == (3, )
  features = [xx ** 0,
              xx, 
              0.5 * xx ** 2]
  MT = np.vstack(features)
  MTM = MT @ MT.T
  pars = np.linalg.lstsq(MTM, MT @ patch.flatten(), rcond=None)[0]
  center = -1.0 * pars[1] / pars[2]
  assert center > np.min(xx)
  assert center < np.max(xx)
  return center

def centroid_max(image):
  assert len(image.shape) == 2
  ismax = np.where(image == np.max(image))
  besty, bestx = ismax[0][0], ismax[1][0]
  bestpatch = image[besty-1:besty+2, bestx-1:bestx+2]
  plt.imshow(bestpatch, interpolation="nearest", cmap="gray")
  ymax = besty + centroid_3(np.sum(bestpatch, axis=1))
  xmax = bestx + centroid_3(np.sum(bestpatch, axis=0))
  return (ymax, xmax)

In [ ]:
# make rectangular data for plotting (and plot along the way)
refimage = imslice(arcs[10][2]) # magic 10
refimage -= np.median(refimage)
plotdata = np.zeros((len(arcs), 4))
with PdfPages('boss_arc_ccf.pdf') as pdf:
  for i, (url, hdr, image) in enumerate(arcs):
    subimage = imslice(image)
    subimage -= np.median(subimage)
    ccf, shifts = cross_correlate(subimage, refimage)
    ymax, xmax = centroid_max(ccf)
    xshift = np.interp(xmax, np.arange(len(shifts)), shifts)
    yshift = np.interp(ymax, np.arange(len(shifts)), shifts)
    fig = plt.figure(figsize=(12, 9))
    plt.imshow(ccf,
               cmap="gray", interpolation="nearest")
    plt.colorbar()
    plt.title(url + " ({},{})".format(yshift, xshift))
    pdf.savefig(fig)
    plt.close(fig)
    plotdata[i] = hdr["ALT"], hdr["AZ"], hdr["IPA"], yshift
labels = ["altitude (deg)", "azimuth (deg)", "position angle (deg)", "dispersion shift (pix)"]

In [ ]:
# make rectangular data for corner plotting
f = corner.corner(plotdata, labels=labels, plot_contours=False)